In [1]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [2]:
dataset = pd.read_csv('../Datasets/Heart Disease/heart.csv')

In [3]:
X, y = dataset.iloc[:, :-1], dataset.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# TRAIN MODEL

In [8]:
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=45)
automl.fit(X_train.values, y_train)
y_hat = automl.predict(X_test.values)

In [9]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.9922178988326849

In [10]:
import pickle
with open('./models/heart_disease_automl2.pkl', 'wb') as f:
    pickle.dump(automl, f)

# LOAD MODEL

In [4]:
import pickle
with open('./models/heart_disease_automl2.pkl', 'rb') as f:
    automl = pickle.load(f)

len(automl.show_models())

12

# LIME

In [16]:
import lime
import lime.lime_tabular
import tqdm 

In [17]:
continuous_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
categorical_features = X_train.columns.drop(continuous_features).tolist()
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns.tolist(), class_names=['No Disease', 'Disease'], categorical_features=categorical_features, discretize_continuous=True)

In [18]:
text_x = X_test.values

In [19]:
exp_fn = lambda i: explainer.explain_instance(X_test.iloc[i], automl.predict_proba, num_features=len(X_test.columns))
def exp_fn_blk(xtest, exp_fn):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = exp_fn(i)
        exp1.append(exp.as_map()[exp.available_labels()[0]])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x, exp_fn))

In [12]:
import metrics

In [24]:
exp1 = exp_fn_wrap(text_x)
exp2 = exp_fn_wrap(text_x)

100%|██████████| 257/257 [16:07<00:00,  3.76s/it]


In [25]:
np.save('./explanations/heart_disease1.npy', exp1)
np.save('./explanations/heart_disease2.npy', exp2)

In [26]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [27]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(X_test.columns))
sb = metrics.calc_stability(enc1, y_test)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [28]:
i, s, sb

((100.0, 0, 257), (0, 257, 66049, 0.0), (66, 257))

In [29]:
X_test_norm = metrics.normalize_test(X_train, X_test)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [30]:
sim

0.29256260533863654

In [31]:
trust = metrics.calc_trust_score(automl, text_x, exp1, 3, X_train.columns.tolist())

100%|██████████| 257/257 [1:01:44<00:00, 14.41s/it]


In [32]:
trust

0.437094682230869

In [33]:
list_monotonicity = []
list_non_sensitivity = []
list_effective_complexity = []

for i in tqdm.tqdm(range(len(X_test))):
    atr = exp1[i]
    sorted_atr = [j for i,j in atr]
    sorted_feat = [i for i,j in atr]
    y = np.zeros(2, dtype=int)
    np.put(y, y_test.iloc[i], 1)
    example = metrics.FeatureAttribution(automl, X_test.to_numpy()[i], y, sorted_atr)
    list_monotonicity.append(example.monotonicity())
    list_non_sensitivity.append(example.non_sensitivity())
    list_effective_complexity.append(example.effective_complexity(sorted_feat, 0.1))

  0%|          | 0/257 [00:00<?, ?it/s]

100%|██████████| 257/257 [21:05<00:00,  4.92s/it]


In [34]:
print(np.mean(list_monotonicity))
print(np.mean(list_non_sensitivity))
print(np.mean(list_effective_complexity))

print(np.median(list_monotonicity))
print(np.median(list_non_sensitivity))
print(np.median(list_effective_complexity))

-0.004596570744430668
0.0
1.821011673151751
-0.021978021978021976
0.0
0.0


# CIU

In [6]:
from ciu import determine_ciu
import tqdm

In [7]:
feat_list = X_train.columns.tolist()

In [8]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = determine_ciu(X_test.iloc[i:i+1], automl.predict_proba, X_train.to_dict('list'), samples = 1000, prediction_index = 1)
        exp_list = [[feat_list.index(i), exp.ci[i]] for i in exp.ci]
        exp1.append(exp_list)
    return np.array(exp1)

In [9]:
exp1 = exp_fn_blk(X_test)
exp2 = exp_fn_blk(X_test)

  0%|          | 0/257 [00:00<?, ?it/s]

  0%|          | 0/257 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [39]:
np.save('./explanations/heart_disease_ciu1.npy', exp1)
np.save('./explanations/heart_disease_ciu2.npy', exp2)

In [10]:
exp1 = np.load('./explanations/heart_disease_ciu1.npy')
exp2 = np.load('./explanations/heart_disease_ciu2.npy')

In [40]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(feat_list))
sb = metrics.calc_stability(enc1, y_test)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [41]:
i, s, sb

((100.0, 0, 257), (0, 257, 66049, 0.0), (46, 257))

In [42]:
X_test_norm = metrics.normalize_test(X_train, X_test)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [43]:
sim

0.5813001331891391

In [44]:
list_monotonicity = []
list_non_sensitivity = []
list_effective_complexity = []

for i in tqdm.tqdm(range(len(X_test))):
    atr = exp1[i]
    sorted_atr = [j for i,j in atr]
    sorted_feat = [i for i,j in atr]
    y = np.zeros(2, dtype=int)
    np.put(y, y_test.iloc[i], 1)
    example = metrics.FeatureAttribution(automl, X_test.to_numpy()[i], y, sorted_atr)
    list_monotonicity.append(example.monotonicity())
    list_non_sensitivity.append(example.non_sensitivity())
    list_effective_complexity.append(example.effective_complexity(sorted_feat, 0.1))

100%|██████████| 257/257 [21:20<00:00,  4.98s/it]


In [45]:
print(np.mean(list_monotonicity))
print(np.mean(list_non_sensitivity))
print(np.mean(list_effective_complexity))

print(np.median(list_monotonicity))
print(np.median(list_non_sensitivity))
print(np.median(list_effective_complexity))

0.12263223158164792
0.0
2.1945525291828796
0.12637362637362637
0.0
0.0


In [13]:
metrics.calc_trust_score(automl, X_test.to_numpy(), exp1, 3, X_train.columns.to_list())

100%|██████████| 257/257 [1:52:15<00:00, 26.21s/it]   


0.4617380025940337

# RULEFIT

In [14]:
from skrules import SkopeRules
import metrics_rules
import time

In [15]:
clf = SkopeRules(max_depth_duplication=2,
                    n_estimators=512,
                    precision_min=0.3,
                    recall_min=0.1,
                    feature_names=X_train.columns.tolist())

In [16]:
start_time = time.time()
clf.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
top_rules1 = clf.score_top_rules(X_test)
top_rules2 = clf.score_top_rules(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.04627823829650879 seconds ---


In [8]:
i = metrics_rules.calc_identity_rules(top_rules1, top_rules2)
print(i)

s = metrics_rules.calc_separability_rules(top_rules1)
print(s)

enc_rules = metrics_rules.exp_enc(clf, top_rules1)
sb = metrics_rules.calc_stability_rules(enc_rules, y_test)
print(sb)

(0.0, 257, 257)
(19630, 257, 66049, 29.72035912731457)
(50, 257)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [9]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test)
sim = metrics_rules.calc_similarity(enc_rules, X_test_norm)
print(sim)

0.0


# RULEMATRIX

In [5]:
continuous_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
categorical_features = X_train.columns.drop(continuous_features).tolist()

In [6]:
import rulematrix
import time
import metrics_rules

In [7]:
is_continuous = [True if i in continuous_features else False for i in X_train.columns.tolist()]
is_categorical = [True if i in categorical_features else False for i in X_train.columns.tolist()]

In [8]:
surrogate = rulematrix.surrogate.rule_surrogate(
    automl.predict,
    X_train,
    sampling_rate=4,
    is_continuous=is_continuous,
    is_categorical=is_categorical,
    seed=42
)

In [9]:
test_x = X_test.to_numpy()

In [10]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in range(len(xtest)):
        queried_rules = np.arange(surrogate.student.n_rules)[surrogate.student.decision_path(test_x[i].reshape(1,-1)).reshape(-1)]
        exp1.append(queried_rules[-1])
    return np.array(exp1)
exp_fn_wrap = lambda x: np.array(exp_fn_blk(x))

In [11]:
start_time = time.time()
exp1 = exp_fn_blk(test_x)
exp2 = exp_fn_blk(test_x)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.17894291877746582 seconds ---


In [12]:
def enc_exp(exp, n_features):
    enc = []
    for i in range(exp.shape[0]):
        new = np.zeros(n_features)
        for j in surrogate.student.rule_list[exp[i]].clauses:
            new[j.feature_idx] = 1
        enc.append(new)
    return np.array(enc)

In [13]:
enc_exp = enc_exp(exp1, X_train.shape[1])

In [14]:
i = metrics_rules.calc_identity_rules(exp1, exp2)
print(i)

s = metrics_rules.calc_separability_rules(exp1)
print(s)

sb = metrics_rules.calc_stability_rules(enc_exp, y_test)
print(sb)

(0.0, 257, 257)


(5108, 257, 66049, 7.733652288452513)
(53, 257)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [15]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test)
sim = metrics_rules.calc_similarity(enc_exp, X_test_norm)

In [16]:
sim

1.243738795275592